# 🌊 DeepSeek Integration i Google Colab

Dette notebook viser hvordan du bruger DeepSeek i Google Colab/Antigravity

---

## 📦 Step 1: Installer Dependencies

In [ ]:
# Install OpenAI SDK (DeepSeek er kompatibel)
!pip install openai --quiet
print("✅ OpenAI SDK installeret!")

## 🔑 Step 2: Konfigurer API Key

In [ ]:
import openai
from openai import OpenAI

# Din DeepSeek API key
DEEPSEEK_API_KEY = "sk-a3f8e6b48271466b981396dc97fd904a"

# Opret client
client = OpenAI(
    api_key=DEEPSEEK_API_KEY,
    base_url="https://api.deepseek.com/v1"
)

print("✅ DeepSeek client konfigureret!")

## 💬 Step 3: Simple Chat

In [ ]:
def chat_with_deepseek(prompt, temperature=0.7, max_tokens=2000):
    """
    Chat med DeepSeek
    """
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    return response.choices[0].message.content

# Test det
response = chat_with_deepseek("Hej DeepSeek! Sig hej på dansk.")
print("🤖 DeepSeek:", response)

## 🎯 Step 4: Praktiske Eksempler

### Eksempel 1: Code Generation

In [ ]:
code_prompt = """
Skriv en Python funktion der:
1. Tager en liste af tal
2. Fjerner duplikater
3. Sorterer listen
4. Returnerer resultat

Inkluder docstring og type hints.
"""

code = chat_with_deepseek(code_prompt, temperature=0.3)
print("💻 Genereret kode:")
print(code)

### Eksempel 2: Data Analysis

In [ ]:
data_prompt = """
Jeg har følgende data om website traffic:
- Mandag: 1500 besøg
- Tirsdag: 1800 besøg
- Onsdag: 1200 besøg
- Torsdag: 2100 besøg
- Fredag: 2500 besøg

Analyser dataen og giv insights på dansk.
"""

analysis = chat_with_deepseek(data_prompt)
print("📊 Analyse:")
print(analysis)

### Eksempel 3: Multi-turn Conversation

In [ ]:
conversation = [
    {"role": "system", "content": "Du er en hjælpsom dansk programmerings-assistent."},
    {"role": "user", "content": "Hvad er forskellen på list og tuple i Python?"}
]

# First message
response1 = client.chat.completions.create(
    model="deepseek-chat",
    messages=conversation
)

answer1 = response1.choices[0].message.content
print("Q1: Hvad er forskellen på list og tuple?")
print("A1:", answer1)
print()

# Add to conversation
conversation.append({"role": "assistant", "content": answer1})
conversation.append({"role": "user", "content": "Kan du give et konkret eksempel?"})

# Second message
response2 = client.chat.completions.create(
    model="deepseek-chat",
    messages=conversation
)

answer2 = response2.choices[0].message.content
print("Q2: Kan du give et konkret eksempel?")
print("A2:", answer2)

## 🚀 Step 5: Advanced - Stream Response

In [ ]:
import sys

def stream_chat(prompt):
    """
    Stream response fra DeepSeek
    """
    stream = client.chat.completions.create(
        model="deepseek-chat",
        messages=[{"role": "user", "content": prompt}],
        stream=True
    )
    
    print("🤖 DeepSeek (streaming): ", end="")
    
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            content = chunk.choices[0].delta.content
            print(content, end="", flush=True)
    
    print()  # New line

# Test streaming
stream_chat("Fortæl mig en kort historie om en robot der lærer at kode.")

## 🎨 Step 6: Interactive Chat Interface

In [ ]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

# Chat history
chat_history = []

# UI Components
output = widgets.Output()
text_input = widgets.Textarea(
    placeholder='Skriv din besked...',
    layout=widgets.Layout(width='100%', height='80px')
)
send_button = widgets.Button(
    description='Send til DeepSeek',
    button_style='success',
    layout=widgets.Layout(width='200px')
)
clear_button = widgets.Button(
    description='Ryd chat',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

def send_message(b):
    user_message = text_input.value
    if not user_message.strip():
        return
    
    # Add user message
    chat_history.append({"role": "user", "content": user_message})
    
    # Get response
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=chat_history
    )
    
    assistant_message = response.choices[0].message.content
    chat_history.append({"role": "assistant", "content": assistant_message})
    
    # Display
    with output:
        clear_output()
        for msg in chat_history:
            if msg["role"] == "user":
                display(Markdown(f"**👤 Du:** {msg['content']}"))
            else:
                display(Markdown(f"**🤖 DeepSeek:** {msg['content']}"))
            print("---")
    
    # Clear input
    text_input.value = ""

def clear_chat(b):
    global chat_history
    chat_history = []
    with output:
        clear_output()
        display(Markdown("*Chat ryddet. Start en ny samtale!*"))

send_button.on_click(send_message)
clear_button.on_click(clear_chat)

# Display UI
display(widgets.VBox([
    widgets.HTML("<h2>🌊 Chat med DeepSeek</h2>"),
    output,
    text_input,
    widgets.HBox([send_button, clear_button])
]))

## 📊 Step 7: Token Usage & Cost Tracking

In [ ]:
def chat_with_stats(prompt):
    """
    Chat og vis token statistik
    """
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[{"role": "user", "content": prompt}]
    )
    
    content = response.choices[0].message.content
    usage = response.usage
    
    # DeepSeek pricing (per 1M tokens)
    input_cost = 0.27
    output_cost = 1.1
    
    total_cost = (
        (usage.prompt_tokens / 1_000_000) * input_cost +
        (usage.completion_tokens / 1_000_000) * output_cost
    )
    
    print("🤖 DeepSeek:", content)
    print()
    print("📊 Statistik:")
    print(f"  Input tokens: {usage.prompt_tokens:,}")
    print(f"  Output tokens: {usage.completion_tokens:,}")
    print(f"  Total tokens: {usage.total_tokens:,}")
    print(f"  💰 Estimated cost: ${total_cost:.6f}")

# Test
chat_with_stats("Forklar machine learning på dansk i 3 sætninger.")

## 🎉 Du er klar!

Nu kan du bruge DeepSeek i Google Colab/Antigravity!

### Næste skridt:
- Eksperimenter med forskellige prompts
- Prøv forskellige temperature settings
- Byg din egen chat application
- Integrer med dine data

**Tips:**
- DeepSeek er særligt god til coding tasks
- Brug lav temperature (0.3) for code generation
- Brug høj temperature (0.8) for kreativt indhold
